<a href="https://colab.research.google.com/github/AndreyBuyanov/HPC.Lab2/blob/main/HPC_Lab2_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'

In [ ]:
![ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'

In [ ]:
!pip install findspark > /dev/null

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("HPC.Lab2").getOrCreate()

In [ ]:
main_df = spark.read.csv(r"./drive/My Drive/TPU/HPC/Lab2/DS_2019_public.csv", header=True, inferSchema=True)
main_df.show()

+------------------+--------+-----------------+--------+-----------+--------+---------+-----------+-----------+-----------+------+-----------+--------+--------+-----+--------+--------+-----------+---------+--------+--------+--------+--------+-------+------+-----------+--------+-----------+-----+--------+--------+--------+--------+---------+------+-----+--------+--------+--------+--------+--------+--------+------+-----------+--------+--------+-------+-----------+--------+--------+------------+---------+---------+---------+-------+----------+--------+--------+-------+--------+--------+--------+---------+------+-------+-------+--------+--------+--------+---------+--------+--------+--------+---------+--------+-------+------------+---------+-----------+-----------+--------+--------+----+------+-------+--------+-----+----+-------+-------+-----------+--------+-------+--------+-----------+---------+--------+---------+-----------+--------+-----+--------+---------+-----+-----+--------+----------

Объявляем константы, выбираем признаки

In [ ]:
MY_VARIANT=2

features = [
            'DIVISION', # Census Division
            'KWHCOL',   # Electricity usage for air-conditioning, central and window/wall (room), in kilowatt-hours
            'NWEIGHT',  # Final sample weight
            'DOLLAREL', # Total Electricity cost, in whole dollars
            'TOTALBTU', # Total usage, in thousand BTU, 2009
            'STORIES',  # Number of stories in a single-family home
            ] 

target = 'Climate_Region_Pub'

Выбираем из исходного датафрейма необходимые столбцы согласно набору признаков.

In [ ]:
from pyspark.sql import functions as F

work_columns = features + [target]
# Выбираем только строки, в которых значение климатического региона совпадает с вариантом задания
#work_df = main_df.where(F.col(target) == MY_VARIANT).select(work_columns)
work_df = main_df.select(work_columns)
work_df.show()

+--------+--------+-----------+--------+--------+-------+------------------+
|DIVISION|  KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|
+--------+--------+-----------+--------+--------+-------+------------------+
|      10| 181.998| 8599.17201|     475|  103460|     31|                 5|
|       1| 184.459|8969.915921|     588|   58716|     -2|                 1|
|       3|1063.022| 18003.6396|     952|   76401|     10|                 1|
|       1|     0.0|5999.605242|     705|   59809|     20|                 1|
|       4|  274.53|4232.486778|    1050|  114350|     20|                 1|
|       2|1224.714|7862.341967|    2444|  150726|     20|                 4|
|       5|6129.163|6297.038285|    2622|   78230|     10|                 3|
|       2| 401.146|12156.72151|    1240|   52677|     20|                 4|
|       4|1080.527|3242.224473|     655|   69166|     10|                 4|
|      10|     0.0|8812.434127|     373|   46796|     -2|                 2|

Отобразим схему датасета.

In [ ]:
work_df.printSchema()

root
 |-- DIVISION: integer (nullable = true)
 |-- KWHCOL: double (nullable = true)
 |-- NWEIGHT: double (nullable = true)
 |-- DOLLAREL: integer (nullable = true)
 |-- TOTALBTU: integer (nullable = true)
 |-- STORIES: integer (nullable = true)
 |-- Climate_Region_Pub: integer (nullable = true)



Добавляем в датасет столбец, значения которого - это вектор, содержащий элементы из столбцов, соответствующим набору признаков (по сути просто значеия столбцов в соответствующей строке упаковали в массив\вектор). Разделяем получившийся датасет на обучающую и тестовую выборки.

In [ ]:
from pyspark.ml.feature import VectorAssembler

def make_datasets(dataframe, inputCols, outputCol):
    assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)
    assembled_df = assembler.transform(dataframe)
    return assembled_df.randomSplit([0.8, 0.2])

Формируем выборки из исходного датасета.

In [ ]:
train_df, test_df = make_datasets(work_df, features, 'features')

Отобразим обучающую выборку.

In [ ]:
train_df.show(truncate=False)

+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|DIVISION|KWHCOL|NWEIGHT    |DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|features                                  |
+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|1       |0.0   |2886.607847|387     |86552   |10     |1                 |[1.0,0.0,2886.607847,387.0,86552.0,10.0]  |
|1       |0.0   |3004.501263|1805    |34339   |20     |1                 |[1.0,0.0,3004.501263,1805.0,34339.0,20.0] |
|1       |0.0   |3209.586624|501     |74951   |10     |1                 |[1.0,0.0,3209.586624,501.0,74951.0,10.0]  |
|1       |0.0   |3912.873751|1647    |106655  |20     |1                 |[1.0,0.0,3912.873751,1647.0,106655.0,20.0]|
|1       |0.0   |3912.8741  |1203    |102144  |10     |1                 |[1.0,0.0,3912.8741,1203.0,102144.0,10.0]  |
|1       |0.0   |3912.874316|1703    |117719  |20     |1

Отобразим тестовую выборку.

In [ ]:
test_df.show(truncate=False)

+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|DIVISION|KWHCOL|NWEIGHT    |DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|features                                  |
+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|1       |0.0   |3912.874323|1364    |165235  |20     |1                 |[1.0,0.0,3912.874323,1364.0,165235.0,20.0]|
|1       |0.0   |3912.874443|814     |97538   |10     |1                 |[1.0,0.0,3912.874443,814.0,97538.0,10.0]  |
|1       |0.0   |4052.687155|3364    |216037  |20     |1                 |[1.0,0.0,4052.687155,3364.0,216037.0,20.0]|
|1       |0.0   |4088.051126|1124    |104485  |20     |1                 |[1.0,0.0,4088.051126,1124.0,104485.0,20.0]|
|1       |0.0   |4120.26313 |990     |108725  |20     |1                 |[1.0,0.0,4120.26313,990.0,108725.0,20.0]  |
|1       |0.0   |4133.556176|3177    |239094  |20     |1

Создаём экземпляры соответствующих заданию классификаторов.

In [ ]:
from pyspark.ml.classification import (LogisticRegression, RandomForestClassifier)
log_reg = LogisticRegression(featuresCol='features', labelCol=target)
ran_for = RandomForestClassifier(featuresCol='features', labelCol=target)

Объявляем функцию оценки модели.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def evaluate(dataFrame, target):
    evaluator = MulticlassClassificationEvaluator(labelCol=target)
    return evaluator.evaluate(dataFrame)

1. Логистическая регрессия. Обучаем на обучающей выборке, проверяем на тестовой выборке, оцениваем модель.

In [ ]:
fitted_model_log_reg = log_reg.fit(train_df)
log_reg_df = fitted_model_log_reg.transform(test_df)
log_reg_evaluation = evaluate(log_reg_df, target)
print(f"Evaluation = {log_reg_evaluation}")

Evaluation = 0.7403561281444964


Отобразим датафрейм с предсказанными значениями. Видим, что для большинства записей начальное и предсказанное значения совпадают.

In [ ]:
log_reg_df.select(work_columns + ['prediction']).orderBy(F.desc('NWEIGHT')).show()

+--------+--------+-----------+--------+--------+-------+------------------+----------+
|DIVISION|  KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|prediction|
+--------+--------+-----------+--------+--------+-------+------------------+----------+
|       3|     0.0|45644.32013|     865|   67719|     10|                 1|       1.0|
|       8| 208.075|39427.92943|     424|   23961|     10|                 1|       5.0|
|       8| 311.862| 39377.6168|     834|   93277|     20|                 1|       1.0|
|       7|2328.789|38977.82412|    1221|  116253|     10|                 4|       4.0|
|       5|2396.059|37518.82806|    1327|  102132|     20|                 4|       4.0|
|       8| 1054.54|37261.45408|    2173|   95440|     10|                 1|       4.0|
|       7|1401.407|34738.58159|    1521|   62804|     10|                 4|       4.0|
|       6|1376.394|34308.85714|    1936|   51088|     10|                 4|       4.0|
|       8| 712.966|33453.16027| 

Видим, что все пять классов присутствуют.

In [ ]:
log_reg_df.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  864|
|       4.0|  511|
|       3.0|  393|
|       2.0|  318|
|       5.0|  104|
+----------+-----+



Отобразим количество правильных предсказаний по классам.

In [ ]:
from pyspark.sql.types import IntegerType
log_reg_df.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  645|
|       4.0|  375|
|       3.0|  305|
|       2.0|  247|
|       5.0|   61|
+----------+-----+



2. Метод случайного леса. Обучаем на обучающей выборке, проверяем на тестовой выборке, оцениваем модель.

In [ ]:
fitted_model_ran_for = ran_for.fit(train_df)
ran_for_df = fitted_model_ran_for.transform(test_df)
ran_for_evaluation = evaluate(ran_for_df, target)
print(f"Evaluation = {ran_for_evaluation}")

Evaluation = 0.8012654366478017


Отобразим датафрейм с предсказанными значениями. Видим, что для большинства записей начальное и предсказанное значения совпадают.

In [ ]:
ran_for_df.select(work_columns + ['prediction']).orderBy(F.desc('NWEIGHT')).show()

+--------+--------+-----------+--------+--------+-------+------------------+----------+
|DIVISION|  KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|prediction|
+--------+--------+-----------+--------+--------+-------+------------------+----------+
|       3|     0.0|45644.32013|     865|   67719|     10|                 1|       1.0|
|       8| 208.075|39427.92943|     424|   23961|     10|                 1|       1.0|
|       8| 311.862| 39377.6168|     834|   93277|     20|                 1|       1.0|
|       7|2328.789|38977.82412|    1221|  116253|     10|                 4|       4.0|
|       5|2396.059|37518.82806|    1327|  102132|     20|                 4|       4.0|
|       8| 1054.54|37261.45408|    2173|   95440|     10|                 1|       1.0|
|       7|1401.407|34738.58159|    1521|   62804|     10|                 4|       4.0|
|       6|1376.394|34308.85714|    1936|   51088|     10|                 4|       4.0|
|       8| 712.966|33453.16027| 

Видим, что все пять классов присутствуют.

In [ ]:
ran_for_df.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  884|
|       4.0|  482|
|       3.0|  399|
|       2.0|  301|
|       5.0|  124|
+----------+-----+



Отобразим количество правильных предсказаний по классам.

In [ ]:
ran_for_df.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  710|
|       4.0|  415|
|       3.0|  316|
|       2.0|  244|
|       5.0|   79|
+----------+-----+



В исходном датафрейме меняем значение климатического региона на 1, если значение соответствует варианту, 0 - если значение не соответствует варианту. В получившемся датасете будет только два климатических региона. Делаем те же операции, что и с исходным датасетом.

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

udf = UserDefinedFunction(lambda x: 1 if x == MY_VARIANT else 0, IntegerType())
modified_main_df = main_df.select(*[udf(column).alias(column) if column == target else column for column in main_df.columns])
modified_main_df.show()

+------------------+--------+-----------------+--------+-----------+--------+---------+-----------+-----------+-----------+------+-----------+--------+--------+-----+--------+--------+-----------+---------+--------+--------+--------+--------+-------+------+-----------+--------+-----------+-----+--------+--------+--------+--------+---------+------+-----+--------+--------+--------+--------+--------+--------+------+-----------+--------+--------+-------+-----------+--------+--------+------------+---------+---------+---------+-------+----------+--------+--------+-------+--------+--------+--------+---------+------+-------+-------+--------+--------+--------+---------+--------+--------+--------+---------+--------+-------+------------+---------+-----------+-----------+--------+--------+----+------+-------+--------+-----+----+-------+-------+-----------+--------+-------+--------+-----------+---------+--------+---------+-----------+--------+-----+--------+---------+-----+-----+--------+----------

Выбираем из исходного датафрейма необходимые столбцы согласно набору признаков.

In [ ]:
work_df2 = modified_main_df.select(work_columns)
work_df2.show()

+--------+--------+-----------+--------+--------+-------+------------------+
|DIVISION|  KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|
+--------+--------+-----------+--------+--------+-------+------------------+
|      10| 181.998| 8599.17201|     475|  103460|     31|                 0|
|       1| 184.459|8969.915921|     588|   58716|     -2|                 0|
|       3|1063.022| 18003.6396|     952|   76401|     10|                 0|
|       1|     0.0|5999.605242|     705|   59809|     20|                 0|
|       4|  274.53|4232.486778|    1050|  114350|     20|                 0|
|       2|1224.714|7862.341967|    2444|  150726|     20|                 0|
|       5|6129.163|6297.038285|    2622|   78230|     10|                 0|
|       2| 401.146|12156.72151|    1240|   52677|     20|                 0|
|       4|1080.527|3242.224473|     655|   69166|     10|                 0|
|      10|     0.0|8812.434127|     373|   46796|     -2|                 1|

Формируем выборки из модифицированного датасета.

In [ ]:
train_df2, test_df2 = make_datasets(work_df2, features, 'features')

Отобразим обучающую выборку.

In [ ]:
train_df2.show(truncate=False)

+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|DIVISION|KWHCOL|NWEIGHT    |DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|features                                  |
+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|1       |0.0   |2886.607847|387     |86552   |10     |0                 |[1.0,0.0,2886.607847,387.0,86552.0,10.0]  |
|1       |0.0   |3004.501263|1805    |34339   |20     |0                 |[1.0,0.0,3004.501263,1805.0,34339.0,20.0] |
|1       |0.0   |3209.586624|501     |74951   |10     |0                 |[1.0,0.0,3209.586624,501.0,74951.0,10.0]  |
|1       |0.0   |3912.873751|1647    |106655  |20     |0                 |[1.0,0.0,3912.873751,1647.0,106655.0,20.0]|
|1       |0.0   |3912.8741  |1203    |102144  |10     |0                 |[1.0,0.0,3912.8741,1203.0,102144.0,10.0]  |
|1       |0.0   |3912.874323|879     |121534  |20     |0

Отобразим тестовую выборку.

In [ ]:
test_df2.show(truncate=False)

+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|DIVISION|KWHCOL|NWEIGHT    |DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|features                                  |
+--------+------+-----------+--------+--------+-------+------------------+------------------------------------------+
|1       |0.0   |3912.874316|1703    |117719  |20     |0                 |[1.0,0.0,3912.874316,1703.0,117719.0,20.0]|
|1       |0.0   |3912.874323|759     |111740  |20     |0                 |[1.0,0.0,3912.874323,759.0,111740.0,20.0] |
|1       |0.0   |3991.10201 |2291    |111593  |20     |0                 |[1.0,0.0,3991.10201,2291.0,111593.0,20.0] |
|1       |0.0   |4088.051126|1124    |104485  |20     |0                 |[1.0,0.0,4088.051126,1124.0,104485.0,20.0]|
|1       |0.0   |4088.051311|1028    |83883   |20     |0                 |[1.0,0.0,4088.051311,1028.0,83883.0,20.0] |
|1       |0.0   |4120.26313 |990     |108725  |20     |0

1. Логистическая регрессия. Обучаем на обучающей выборке, проверяем на тестовой выборке, оцениваем модель.

In [ ]:
fitted_model_log_reg2 = log_reg.fit(train_df2)
log_reg_df2 = fitted_model_log_reg2.transform(test_df2)
log_reg_evaluation2 = evaluate(log_reg_df2, target)
print(f"Evaluation = {log_reg_evaluation2}")

Evaluation = 0.9169147216541


Отобразим датафрейм с предсказанными значениями. Видим, что для большинства записей начальное и предсказанное значения совпадают.

In [ ]:
log_reg_df2.select(work_columns + ['prediction']).orderBy(F.desc('KWHCOL')).show()

+--------+---------+-----------+--------+--------+-------+------------------+----------+
|DIVISION|   KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|prediction|
+--------+---------+-----------+--------+--------+-------+------------------+----------+
|       5|24678.649|6557.735971|    9316|  235452|     10|                 0|       0.0|
|       9|17878.516| 10550.9008|    4865|  249619|     20|                 1|       1.0|
|       5|15106.364|7190.389659|    6157|  194293|     20|                 0|       0.0|
|       9|15051.023|9539.455973|    4858|  142007|     10|                 1|       1.0|
|       7|15021.175|8345.547105|    4057|  203249|     20|                 0|       0.0|
|       5|14246.889|6557.736675|    3684|  111313|     20|                 0|       0.0|
|       9|13804.664|9539.455514|    3767|  121188|     10|                 1|       1.0|
|       7| 13764.18|7604.878355|    5109|  133684|     10|                 0|       0.0|
|       9|12992.765|9

In [ ]:
log_reg_df2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1759|
|       1.0|  371|
+----------+-----+



Отобразим количество правильных предсказаний по классам.

In [ ]:
log_reg_df2.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1688|
|       1.0|  261|
+----------+-----+



2. Метод случайного леса. Обучаем на обучающей выборке, проверяем на тестовой выборке, оцениваем модель.

In [ ]:
fitted_model_ran_for2 = ran_for.fit(train_df2)
ran_for_df2 = fitted_model_ran_for2.transform(test_df2)
ran_for_evaluation2 = evaluate(ran_for_df2, target)
print(f"Evaluation = {ran_for_evaluation2}")

Evaluation = 0.9459093717138946


Отобразим датафрейм с предсказанными значениями. Видим, что для большинства записей начальное и предсказанное значения совпадают.

In [ ]:
ran_for_df2.select(work_columns + ['prediction']).orderBy(F.desc('KWHCOL')).show()

+--------+---------+-----------+--------+--------+-------+------------------+----------+
|DIVISION|   KWHCOL|    NWEIGHT|DOLLAREL|TOTALBTU|STORIES|Climate_Region_Pub|prediction|
+--------+---------+-----------+--------+--------+-------+------------------+----------+
|       5|24678.649|6557.735971|    9316|  235452|     10|                 0|       0.0|
|       9|17878.516| 10550.9008|    4865|  249619|     20|                 1|       1.0|
|       5|15106.364|7190.389659|    6157|  194293|     20|                 0|       0.0|
|       9|15051.023|9539.455973|    4858|  142007|     10|                 1|       1.0|
|       7|15021.175|8345.547105|    4057|  203249|     20|                 0|       0.0|
|       5|14246.889|6557.736675|    3684|  111313|     20|                 0|       0.0|
|       9|13804.664|9539.455514|    3767|  121188|     10|                 1|       1.0|
|       7| 13764.18|7604.878355|    5109|  133684|     10|                 0|       0.0|
|       9|12992.765|9

In [ ]:
ran_for_df2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1801|
|       1.0|  329|
+----------+-----+



Отобразим количество правильных предсказаний по классам.

In [ ]:
ran_for_df2.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1742|
|       1.0|  273|
+----------+-----+



Используя ранее обученные на исходном датасете модели, предскажем климатический регион, используя модифицированный датасет.

Логистическая регрессия.

In [ ]:
log_reg_df3 = fitted_model_log_reg.transform(test_df2)
log_reg_evaluation3 = evaluate(log_reg_df3, target)
print(f"Evaluation = {log_reg_evaluation3}")
log_reg_df3.groupBy('prediction').count().show()
log_reg_df3.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

Evaluation = 0.0014208618468636186
+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  765|
|       4.0|  531|
|       3.0|  384|
|       2.0|  351|
|       5.0|   99|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|    5|
+----------+-----+



Метод случайного леса.

In [ ]:
ran_for_df3 = fitted_model_ran_for.transform(test_df2)
ran_for_evaluation3 = evaluate(ran_for_df3, target)
print(f"Evaluation = {ran_for_evaluation3}")
ran_for_df3.groupBy('prediction').count().show()
ran_for_df3.where(F.col('Climate_Region_Pub') == F.col('prediction')).groupBy('prediction').count().show()

Evaluation = 0.0008448159598966882
+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|  775|
|       4.0|  509|
|       3.0|  398|
|       2.0|  350|
|       5.0|   98|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|    3|
+----------+-----+

